In [2]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torchdiffeq

from datetime import datetime

#cuda.select_device(0)
#cuda.close()

# Neural-ODE
#os.chdir("./tfdiffeq-master")
#import tfdiffeq as ODE
#import tfdiffeq.models as ODEModels
#os.chdir("../")
torch.cuda.is_available()

False

In [2]:
lookback = 48 * 2
interval = 48

def GHI(data) :
    
    d = data["Day"] % 365
    longitude  = 37.532600
    Tutc = 9
    LSTM = 15 * Tutc
    B = (360 / 365) * (d - 81)
    EoT = 9.87 * np.sin(2 * B) - 7.53 * np.cos(B) - 1.5 * np.sin(B)
    TC = 4 * (127.024612 - LSTM) + EoT
    LST = data["Hour"] + (TC / 60)
    LST[np.array(range(1, LST.shape[0], 2))] += 0.7
    HRA = 15 * (LST- 12)

    theta = np.arcsin(np.sin(23.45) * np.sin(B))
    alpha = np.arcsin(np.sin(theta) * np.sin(longitude) + np.cos(theta) * np.cos(longitude) * np.cos(HRA))
    Zenith = 90 - alpha
    GHI = data["DHI"] + (data["DNI"] * Zenith)
    data.loc[:, "GHI"] = GHI
    
    return

def applyFeatureEngineering(data) :
    
    VaporTable = pd.DataFrame(np.array([0.9673, 1.0595, 1.1596, 1.2679, 1.3856, 1.5133,
                                    1.6517, 1.8010, 1.9632, 2.1379, 2.3267, 2.5314, 2.7516, 2.9888, 3.2444, 3.5199, 3.8167, 4.1352, 4.4777, 
                                    4.8467, 5.1917, 5.5579, 5.9466, 6.3588, 6.7958, 7.2589, 7.7494, 8.2686, 8.8171, 9.3977, 
                                    10.0110, 10.6590, 11.3430, 12.0650, 12.8270, 13.6290, 14.4750, 15.3660, 15.5620, 17.2910, 18.3290, 19.4210,
                                    20.5680, 21.7730, 23.0380, 24.3660, 25.7660, 27.2210, 28.7560, 30.3580, 32.0390, 33.7990, 35.6410, 37.5680, 39.5830]),
                          index = list(range(-19, 36)))

    data.loc[:, "DNI"] = np.log(data["DNI"] + 1)
    data.loc[:, "DHI"] = np.log(data["DHI"] + 1)
    data.loc[:, "WS"] = np.log(data["WS"] + 1)
    GHI(data)
    data.loc[:, "RH"] /= 100
    #data.loc[:, "Vap"] = VaporTable.loc[data["T"]].values * data["RH"].values.reshape(-1, 1)
    formula = np.log(data["RH"]) + (17.62 * data["T"]) / (243.12 + data["T"])
    data.loc[:, "DP"] = (243.12 * formula) / (17.62 - formula)
    data.loc[:, "Dew"] = data["T"] - data["DP"]
    data.loc[:, "WT"] = data["T"] - (data["Dew"] / 3)
    data = data[["Day", "Hour", "Minute", "DHI", "DNI", "RH", "WS", "T", "GHI", "DP", "Dew", "WT", "TARGET"]]
    data = pd.concat([data, data.rolling(window = interval, min_periods = 1).mean().drop(["Day", "Hour", "Minute"], axis = "columns").add_suffix("_AVG")], axis = 1)    
    return data



In [3]:
class DecimalScaler :
    def __init__(self) :
        self.dec = 0
        return
    
    def calculate(self, data) :
        return data / self.dec
    
    def fit(self, data) :
        self.dec = 10 ** np.array([len(i.split('.')[0]) for i in np.floor(data.max()).astype(str)])
        return
        
    def fit_transform(self, data) :
        self.fit(data)
        return self.calculate(data)
    
    def transform(self, data) :
        return self.calculate(data)
    
    def inverse_transform(self, data) :
        return data * self.dec

In [4]:
train = pd.read_csv("./RawData/train/train.csv")
train = applyFeatureEngineering(train)


columnsIndex = np.array(range(train.columns.values.shape[0]))[np.isin(train.columns.values, ["DHI", "DNI", "RH", "GHI", "WS", "TARGET"])]#, "DHI_AVG", "DNI_AVG", "RH_AVG", "GHI_AVG", "WS_AVG", "TARGET_AVG"])]

trainFeature_2d = pd.DataFrame(train.iloc[:48 * 48 * 15, columnsIndex].reset_index(drop = True))
trainTarget_2d = train["TARGET"].iloc[:48 * 48 * 15]
trainTarget_2d = pd.DataFrame(trainTarget_2d.values.reshape(-1, 1), index = trainFeature_2d.index)

validFeature_2d = pd.DataFrame(train.iloc[48 * 48 * 15:, columnsIndex].reset_index(drop = True))
validTarget_2d = train["TARGET"].iloc[48 * 48 * 15:]
validTarget_2d = pd.DataFrame(validTarget_2d.values.reshape(-1, 1), index = validFeature_2d.index)

featureScaler = DecimalScaler()
targetScaler = DecimalScaler()

trainFeature_2d = pd.DataFrame(featureScaler.fit_transform(trainFeature_2d), index = trainFeature_2d.index)
trainTarget_2d = pd.DataFrame(targetScaler.fit_transform(trainTarget_2d), index = trainFeature_2d.index)

validFeature_2d = pd.DataFrame(featureScaler.transform(validFeature_2d), index = validFeature_2d.index)
validTarget_2d = pd.DataFrame(targetScaler.transform(validTarget_2d), index = validFeature_2d.index)

def makeTimeData(feature, target, timeStep, slideSize) :
    """
    timeStep  : How many "times" will LSTM look back? (Standard : 30 minutes)
                if timeStep value is 336, LSTM will predict target value with past 7 days
    slideSize : Period of standard in time series
    """
    
    newFeature = np.empty(shape = ((feature.shape[0] - (timeStep + 96)) // slideSize, timeStep, feature.shape[1]))
    #newTarget = np.empty(shape = ((feature.shape[0] - (timeStep + 96)) // slideSize, 96, 1))
    newTarget = np.empty(shape = ((feature.shape[0] - (timeStep + 96)) // slideSize, 96))

    for i in range(newFeature.shape[0]) :
        newFeature[i] = feature.iloc[(i * slideSize) : (i * slideSize) + timeStep, :]
        newTarget[i] = target.iloc[(i * slideSize) + timeStep : (i * slideSize) + timeStep + 96].values.reshape(-1)

#    newFeature = tf.convert_to_tensor(newFeature)
#    newTarget = tf.convert_to_tensor(newTarget)
        
    return newFeature, newTarget

trainFeature, trainTarget = makeTimeData(trainFeature_2d, trainTarget_2d, lookback, interval)
validFeature, validTarget = makeTimeData(validFeature_2d, validTarget_2d, lookback, interval)

trainTarget_2d = trainTarget_2d.shift(-96).dropna().reset_index(drop = True)
validTarget_2d = validTarget_2d.shift(-96).dropna().reset_index(drop = True)

trainFeature_2d = trainFeature_2d.iloc[:trainFeature_2d.shape[0] - 96, :].reset_index(drop = True)
validFeature_2d = validFeature_2d.iloc[:validFeature_2d.shape[0] - 96, :].reset_index(drop = True)

In [5]:
class Learner(torchdyn.pl.LightningModule):
    def __init__(self, model:torch.nn.Module):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = torch.nn.L1Loss(y_hat, y)
        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return trainLoader

In [30]:
device = torch.device("cuda:0")

tempTrainFeature_2d = torch.Tensor(trainFeature_2d.values).to(device)
tempTrainTarget_2d = torch.LongTensor(trainTarget_2d.values).to(device)
train = torch.utils.data.TensorDataset(tempTrainFeature_2d, tempTrainTarget_2d)
trainLoader = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = False)

tempValidFeature_2d = torch.Tensor(validFeature_2d.values).to(device)
tempValidTarget_2d = torch.Tensor(validTarget_2d.values).to(device)

valid = torch.utils.data.TensorDataset(tempValidFeature_2d, tempValidTarget_2d)
validLoader = torch.utils.data.DataLoader(valid, batch_size = 32, shuffle = False)


f = torch.nn.Sequential(
    torch.nn.Linear(32, 32),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 32)
    
)

model = torchdyn.NeuralDE(f, sensitivity = "adjoint", solver = "dopri5").to(device)

In [32]:
learn = Learner(model)
trainer = torchdyn.pl.Trainer(min_epochs = 50, max_epochs = 100)
trainer.fit(learn)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name  | Type     | Params
-----------------------------------
0 | model | NeuralDE | 2.1 K 
-----------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params


Epoch 0:   0%|          | 0/1077 [00:00<?, ?it/s] 


RuntimeError: mat1 dim 1 must match mat2 dim 0